In [1]:
import numpy as np
import scipy as sci
import matplotlib.pyplot as plt
from matplotlib import colors
import ipywidgets as widgets
from IPython.display import display
from IPython.display import clear_output  # clear_output(wait=True)
import time
import os
import sys
import h5py

# os.chdir('D:/Documents/PostDoc_KTH/WACQT_Project/ScrptCodes') # on Laptop
os.chdir('../../')  # on Desktop
sys.path.append(os.getcwd())
import myPy_functions as my

#Frequency comb and JPA parameters
f0=4.2           #[GHz] Resonance frequency of the JPA
omega0=2*np.pi*f0
Delta_f=0.1e-3   #[GHz] Modes spacing (typically 100KHz)
Delta=2*np.pi*Delta_f 
c = 299_792_458  # [m/s]


#Style
plt.rcParams['axes.formatter.useoffset'] = False
plt.rcParams['figure.constrained_layout.use'] = True
plt.rcParams['axes.labelsize'] = 20
plt.rcParams['font.size'] = 20
plt.rcParams['legend.fontsize'] = 20
plt.rcParams['legend.frameon'] = False
plt.rcParams["font.weight"] = "normal"
plt.rcParams["font.family"] = "serif"
plt.rcParams["mathtext.fontset"] = "cm"

# Load Exp Data
def load_ExpData(file_, idx_str_):
    # Open hdf5 file
    with h5py.File(file_, "r") as dataset:
        # Data
        return dict(freq_comb=np.asarray(dataset[idx_str_]["freq comb"]),
                    freq_pump=np.asarray(dataset[idx_str_]["freq pumps"]),
                    pump_pwr_data=np.asarray(dataset[idx_str_]["pump pwr sweep"]),
                    pump_phase_data=np.asarray(dataset[idx_str_]["pump phase sweep"]),
                    df=np.asarray(dataset[idx_str_]["df"]),
                    USB=np.asarray(dataset[idx_str_]["USB"]),
                    )
    


In [2]:
#New Data three pumps
file = r'I:/JPA-Data/2025-07/2025-07.hdf5'
cooldown = r'Scattering'
run = '2025-06-19_16_30_23' # 4pumps [-9 -1 1 9], phases [pi 0 0 0] - df [1M..100k], and pump amplitude sweep
run = '2025-06-19_17_01_33' # 4pumps [-9 -1 1 9], phases [pi 0 0 0] - df [1M..100k], and pump amplitude sweep
run = '2025-06-23_12_12_08' # 4pumps [-9 -1 1 9], phases [pi 0 0 0] - df [1M..10k], and pump amplitude sweep
run = '2025-06-24_09_26_33' # 4pumps [-9 -1 1 9], phases [pi 0 0 0] - df [1M..1k], and pump amplitude sweep
# run = '2025-07-12_15_39_29' # Test Npumps, img, df=[100K, 50k, 10k], and pump amplitude sweep
# run = '2025-07-12_18_26_11' # Test Npumps, img, df=[200k, 100K, 50k], Npix20k and pump amplitude sweep
# run = '2025-07-13_00_33_45' # Test Npumps, img, df=[200k, 100K, 50k], Npix20k and pump amplitude sweep

idx_str = "{}/{}".format(cooldown, run)
print(idx_str)
USB = load_ExpData(file, idx_str)["USB"]
df = load_ExpData(file, idx_str)["df"]
Sexp_tot = USB[:, :, 0, :, :]
shapevec = Sexp_tot.shape
# print(shapevec)
Ndf = shapevec[0]
Ng = shapevec[1]
Nphases = shapevec[2]
Nmodes = shapevec[3]

# # Test
# Ndf = 5
# Ng = 11
# Nphases = 1
# Nmodes = 81
# # Sexp_tot = np.zeros((Ndf,Ng,Nphases,Nmodes,Nmodes))
# Sexp_tot = np.random.rand(Ndf,Ng,Nphases,Nmodes,Nmodes)
modes_freq,modes_labels,allmodes_freq = my.generate_freq_vectors(Nmodes,omega0,Delta)
modes_freq_GHz = modes_freq / (2 * np.pi )  # Convert to GHz
# print("Modes frequency vector: ", modes_freq)
# print("Modes labels vector: ", modes_labels)

# Compute reference
S0exp_ref=np.zeros(Ndf)
backgnd_dB=np.zeros(Ndf)
for df_idx in range(Ndf):
    Sref_T = np.squeeze(Sexp_tot[df_idx, 0, :, :])  # Select case 0 power and zero phase as a reference case
    Sref=np.transpose(Sref_T)
    S0exp_ref[df_idx] = np.mean(np.abs(np.diag(Sref)))  # Compute the mean of the diagonal element
    background = np.mean(np.abs(Sref[0:20, 30:60]))
    backgnd_dB[df_idx] = my.dB(background / S0exp_ref[df_idx])

global Sexp_tot
global S0exp_ref
global backgnd_dB
global Ndf
global Ng
global Nphases
global Nmodes
global df


Scattering/2025-06-24_09_26_33


KeyError: "Unable to synchronously open object (object '2025-06-24_09_26_33' doesn't exist)"

In [ ]:
def stuff_to_run_and_display(df_value,g_value,phNoise_value):
    df_idx = df_value
    g_idx = g_value
    phNoise_dB = phNoise_value
    
    global Sexp_tot
    global S0exp_ref
    global backgnd_dB
    global Ndf
    global Ng
    global Nphases
    global Nmodes
    global df

    Sexp_T = np.squeeze(Sexp_tot[df_idx, g_idx, :, :])
    Sexp=np.transpose(Sexp_T)
    Sexp_abs = np.abs(Sexp)
    Sexp_abs_dB = my.dB(Sexp_abs/S0exp_ref[df_idx])
    # Masking phase noise
    eps_dB = phNoise_dB
    mask = Sexp_abs_dB < eps_dB
    # mask = np.where(Sexp_abs_dB < eps_dB)
    Sexp_new = Sexp.copy()  # Create a copy of Sexp to modify
    # Set very small elements exactly to phase zero to suppress numerical phase noise
    Sexp_new[mask] = Sexp_abs[mask] 
    Sexp_abs = np.abs(Sexp_new)
    Sexp_ang = np.angle(Sexp_new)

    # Numerical Scattering matrix
    gp_num = np.linspace(0.0,1.5,Ng)
    # print(gp_num)
    Saa, S0num_ref =my.make_bigS_4pmp_g(Nmodes,gp_num[g_idx],np.pi)
    # Extract small S
    Snum = my.extract_small_S(Saa)
    Snum_abs = np.abs(Snum)
    Snum_ang = np.angle(Snum)
    bckgnd_dB = -60

    ### Plotting
    print('df=', df[df_idx]/1e3, 'kHz')

    fig0, ax0 = plt.subplots(2,2, figsize=(15,10), layout="constrained")
    top=modes_labels[Nmodes-1]
    #|Sexp|
    pl01 = ax0[0,0].imshow(my.dB(Sexp_abs/S0exp_ref[df_idx]), cmap='RdBu_r', extent=[-top, top, top, -top],vmin = -5, vmax = 10)
    plt.colorbar(pl01, ax=ax0[0,0], label='Amplitude [dB]')
    ax0[0,0].set_title('$|S_{exp}|$ ')
    
    #∠Sexp
    pl01 = ax0[0,1].imshow(Sexp_ang/np.pi, cmap='RdBu_r', extent=[-top, top, top, -top],vmin = -1, vmax = 1)
    plt.colorbar(pl01, ax=ax0[0,1], label='Phase [π]')
    ax0[0,1].set_title('$\\angle S_{exp}$ ')

    #|Snum|
    pl01 = ax0[1,0].imshow(my.dB(Snum_abs/S0num_ref + 10**(bckgnd_dB/20)), cmap='RdBu_r', extent=[-top, top, top, -top],vmin = -60, vmax = 10)
    plt.colorbar(pl01, ax=ax0[1,0], label='Amplitude [dB]')
    ax0[1,0].set_title('$|S_{num}|$ ')
    
    #∠Snum
    pl01 = ax0[1,1].imshow(Snum_ang/np.pi, cmap='RdBu_r', extent=[-top, top, top, -top],vmin = -1, vmax = 1)
    plt.colorbar(pl01, ax=ax0[1,1], label='Phase [π]')
    ax0[1,1].set_title('$\\angle S_{num}$ ')

    return


    
# Main Code:
df_slider=widgets.IntSlider(min=0,max=Ndf-1,step=1,value=0,description='df')
g_slider=widgets.IntSlider(min=0,max=Ng-1,step=1,value=0,description='g')
phNoise_slider = widgets.FloatSlider(min=-60.0, max=10.0, step=0.1, value=-35, description='$\phi$ noise [dB]')
# phi_slider=widgets.IntSlider(min=0,max=Ng-1,step=1,value=1,description='$\phi$')
# phi1_slider = widgets.FloatSlider(min=0.0,max=2*np.pi, step=0.01, value=0,description='$\phi_{1}$')

widgets.interact(stuff_to_run_and_display, df_value = df_slider, g_value = g_slider, phNoise_value = phNoise_slider)
                                          

plt.show()     

interactive(children=(IntSlider(value=0, description='df', max=2), IntSlider(value=0, description='g', max=10)…

In [ ]:
# Change of basis Transformation Matrices: a,a* --U--> x,p
U_2by2= (1/np.sqrt(2))*np.array([[1,  1j],
                                 [1, -1j]])                 # \vec{a} = U\vec{x}
Udag_2by2= (1/np.sqrt(2))*np.array([[ 1,  1 ],
                                    [-1j, 1j]])             # \vec{x} = Udag\vec{a}
U=np.kron(np.eye(Nmodes,dtype=complex),U_2by2)              # nmodes is number of repeated blocks
Udag=np.linalg.inv(U) 


def stuff_to_run_and_display(df_value,g_value,phNoise_value):
    df_idx = df_value
    g_idx = g_value
    phNoise_dB = phNoise_value
    
    global Sexp_tot
    global S0exp_ref
    global backgnd_dB
    global Ndf
    global Ng
    global Nphases
    global Nmodes
    global df

    Sexp = np.squeeze(Sexp_tot[df_idx, g_idx, :, :])
    Sexp_abs = np.abs(Sexp)
    Sexp_abs_dB = my.dB(Sexp_abs/S0exp_ref[df_idx])
    # Masking phase noise
    eps_dB = phNoise_dB
    mask = Sexp_abs_dB < eps_dB
    # mask = np.where(Sexp_abs_dB < eps_dB)
    Sexp_new = Sexp.copy()  # Create a copy of Sexp to modify
    # Set very small elements exactly to phase zero to suppress numerical phase noise
    Sexp_new[mask] = Sexp_abs[mask] 
    Sexp_abs = np.abs(Sexp_new)
    Sexp_ang = np.angle(Sexp_new)

    # Numerical Scattering matrix
    gp_num = np.linspace(0.0,1.5,Ng)
    # print(gp_num)
    Saa, S0num_ref =my.make_bigS_4pmp_g(Nmodes,gp_num[g_idx],np.pi)
    # Extract small S
    Snum = my.extract_small_S(Saa)
    Snum_abs = np.abs(Snum)
    Snum_ang = np.angle(Snum)
    bckgnd_dB = -60

    #Compute experimental Big S matrix in a,a* basis
    Sexp_new_cc = Sexp_new.conjugate()  # Complex conjugate
    Saa_exp = np.zeros_like(Saa)
    Saa_exp[::2, ::2] = Sexp_new        # Insert Sexp_new into even indexes 
    Saa_exp[1::2, 1::2] = Sexp_new_cc   # Insert conjugate of Sexp_new into odd indexes
    
    #Compute S matrix in x,p basis
    Sxp_num = (Udag.dot(Saa/S0num_ref).dot(U))
    Sxp_num_T = np.transpose(Sxp_num)
    Sxp_exp = (Udag.dot(Saa_exp/S0exp_ref[df_idx]).dot(U))
    Sxp_exp_T = np.transpose(Sxp_exp)

    # Computing CoVariance matrix
    V0=np.eye(Nmodes*2, dtype=complex)  # Identity matrix as input
    Vnum = np.real(Sxp_num.dot(V0).dot(Sxp_num_T)) 
    Vexp = np.real(Sxp_exp.dot(V0).dot(Sxp_exp_T)) 
    scale_exp=np.max(Vexp)
    scale_num=np.max(Vnum)

    ### Plotting
    print('df=', df[df_idx]/1e3, 'kHz')

    fig0, ax0 = plt.subplots(2,2, figsize=(15,10), layout="constrained")
    top=modes_labels[Nmodes-1]
    #Vexp
    pl01 = ax0[0,0].imshow(Vexp, cmap='RdBu_r', extent=[-top, top, top, -top],vmin=-scale_exp, vmax=scale_exp)
    plt.colorbar(pl01, ax=ax0[0,0], label='V [n]')
    ax0[0,0].set_title('$V_{exp}$ ')
    
    #Vnum
    pl01 = ax0[0,1].imshow(Vnum, cmap='RdBu_r', extent=[-top, top, top, -top],vmin=-scale_num, vmax=scale_num)
    plt.colorbar(pl01, ax=ax0[0,1], label='V [n]')
    ax0[0,1].set_title('$V_{num}$ ')

    #∠Sexp
    pl01 = ax0[1,0].imshow(Sexp_ang/np.pi, cmap='RdBu_r', extent=[-top, top, top, -top],vmin = -1, vmax = 1)
    plt.colorbar(pl01, ax=ax0[1,0], label='Phase [π]')
    ax0[1,0].set_title('$\\angle S_{exp}$ ')
    
    #∠Snum
    pl01 = ax0[1,1].imshow(Snum_ang/np.pi, cmap='RdBu_r', extent=[-top, top, top, -top],vmin = -1, vmax = 1)
    plt.colorbar(pl01, ax=ax0[1,1], label='Phase [π]')
    ax0[1,1].set_title('$\\angle S_{num}$ ')
    return

    
# Main Code:
df_slider=widgets.IntSlider(min=0,max=Ndf-1,step=1,value=0,description='df')
g_slider=widgets.IntSlider(min=0,max=Ng-1,step=1,value=0,description='g')
phNoise_slider = widgets.FloatSlider(min=-60.0, max=10.0, step=0.1, value=-35, description='$\phi$ noise [dB]')
# phi_slider=widgets.IntSlider(min=0,max=Ng-1,step=1,value=1,description='$\phi$')
# phi1_slider = widgets.FloatSlider(min=0.0,max=2*np.pi, step=0.01, value=0,description='$\phi_{1}$')

widgets.interact(stuff_to_run_and_display, df_value = df_slider, g_value = g_slider, phNoise_value = phNoise_slider)
                                          

plt.show()   

interactive(children=(IntSlider(value=0, description='df', max=2), IntSlider(value=0, description='g', max=10)…